In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
pd.set_option('display.max_rows', 1000)



In [79]:
wellcome = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding='latin1')

In [80]:
wellcome1 = wellcome

capjournaltitles = wellcome1['Journal title'].apply(lambda x: str(x).upper())
jtstrip = capjournaltitles.str.strip()
wellcome1['Journal title'] = jtstrip

# wellcome1.head(10)

In [81]:
# Collecting journal articles with alternate journal titles under their appropriate journal

wellcome1['Journal title'] = wellcome1['Journal title'].replace(['PLOSONE', 'PLOS ONE', 'PLOS 1', 'PLOS'], 'PLOS ONE') 
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['ACTA D', 'ACTA CRYSTALLOGRAPHICA SECTION D', 'ACTA CRYSTALLOGRAPHY D', 'ACTA CRYSTALLOGRAPHICA, SECTION D', 'ACTA CRYSTALLOGRAPHICA SECTION D, BIOLOGICAL CRYSTALLOGRAPHY'], 'ACTA CRYSTALLOGRAPHICA SECTION D: BIOLOGICAL CRYSTALLOGRAPHY') 
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['AMERICAN JNL EPIDEMIOLOGY'], 'AMERICAN JOURNAL OF EPIDEMIOLOGY') 
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['AMERICAN JOURNAL OF MEDICAL GENETICS PART A'], 'AMERICAN JOURNAL OF MEDICAL GENETICS') 
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['ANTIMICROBIAL AGENTS AND CHEMOTHERAPY', 'ANTIMICROBIAL AGFENTS AND CHEMOTHERAPY'], 'ANTIMICROBIAL AGENTS & CHEMOTHERAPY') 
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['ANGEWANDE CHEMIE', 'ANGEWANDTE CHEMIE INTERNATIONAL EDITION','ANGEW CHEMS INT ED' ], 'ANGEWANDTE CHEMIE') 
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['BEHAVIOUR RESEARCH AND THERAPY'], 'BEHAVIOR RESEARCH & THERAPY') 
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['BIOCHEM JOURNAL', 'BIOCHEMICAL JOURNALS'], 'BIOCHEMICAL JOURNAL') 
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['BIOCHEM SOC TRANS'], 'BIOCHEMICAL SOCIETY TRANSACTIONS')
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF AMERICA', 'PNAS (PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE UNITED STATES OF AMERICA', 'PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES (PNAS)', 'PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF THE USA', 'NATIONAL ACADEMY OF SCIENCES', 'PROCEDDINGS OF THE NATIONAL ACADEMY OF SCIENCES OF USA', 'PNAS', 'PNAS USA', 'PROC NATL ACAD SCI U S A.'], 'PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES')
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['NUCLEIC ACID RESEARCH'], 'NUCLEIC ACIDS RESEARCH')
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['THE AMERICAL JOURNAL OF HUMAN GENETICS', 'THE AMERICAN JOURNAL OF HUMAN GENETICS'], 'AMERICAN JOURNAL OF HUMAN GENETICS')
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['NEUROIMAGE: CLINICAL', 'NEUROLMAGE'], 'NEUROIMAGE')
wellcome1['Journal title'] = wellcome1['Journal title'].replace(['THE JOURNAL OF BIOLOGICAL CHEMISTRY', 'JOURNAL BIOLOGICAL CHEMISTRY', 'BIOLOGICIAL CHEMISTRY', 'J BIOL CHEM.', 'J BIOL CHEMISTRY', 'JOURNAL OF BIOL CHEM', 'BIOLOGICAL CHEMISTRY', 'JNL BIOLOGICAL CHEMISTRY'], 'JOURNAL OF BIOLOGICAL CHEMISTRY')


# and then calculating the top 5 journals by title count

print(wellcome1['Journal title'].value_counts().head(5))



PLOS ONE                                           210
JOURNAL OF BIOLOGICAL CHEMISTRY                     73
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES     40
NEUROIMAGE                                          36
NUCLEIC ACIDS RESEARCH                              29
Name: Journal title, dtype: int64


In [82]:
# removing the £ and $ from the cost column
remove_poundsign = wellcome1['Cost'].apply(lambda x: str(x).replace('£',''))

wellcome1['Cost'] = remove_poundsign

remove_dollarsign = wellcome1['Cost'].apply(lambda x: str(x).replace('$',''))

wellcome1['Cost'] = remove_dollarsign

In [83]:
# converting the cost to numeric values
wellcome1.Cost = pd.to_numeric(wellcome1.Cost, errors='coerce')

In [84]:
# Checking to see what outliers are present, both above and below the range.
# wellcome1[wellcome1['Cost']>4000]
# wellcome1[wellcome1['Cost']<100]


In [85]:
# removing lines with costs of 999,999
remove_cost9 = wellcome1[wellcome1.Cost != [999999]]
wellcome_costcalc = remove_cost9


# cleaning some of the higher (>10,000) cost values by multiplying them by .01; 
# the assumption here is that they were mistakenly punched in without differentiating between pounds and pence.
wellcome_costs = wellcome_costcalc['Cost']
mod_costs = []
for x in wellcome_costs:
    if x >10000:
        mod_costs.append(x * .01)
    else:
        mod_costs.append(x)
wellcome_costs = mod_costs
wellcome_costcalc['Cost'] = wellcome_costs



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [86]:
# Checking again to see what outliers are present, both above and below the range:
# wellcome_costcalc[wellcome_costcalc['Cost']>4000]
# wellcome_costcalc[wellcome_costcalc['Cost']<100]

# Everything looks within reasonable ranges.

In [87]:
# describing the statistical distribution of cost for each journal title:

costcalc = wellcome_costcalc.groupby('Journal title').describe().reset_index()
costcalc.head(10)

Journal title  Cost             \
                                                     count       mean   
0                 ACADEMY OF NUTRITION AND DIETETICS   1.0  2379.5400   
1                               ACS CHEMICAL BIOLOGY   5.0  1418.1860   
2                          ACS CHEMICAL NEUROSCIENCE   1.0  1186.8000   
3                                           ACS NANO   2.0   668.1400   
4  ACTA CRYSTALLOGRAPHICA SECTION D,  BIOLOGICAL ...   1.0   771.4200   
5  ACTA CRYSTALLOGRAPHICA SECTION D: BIOLOGICAL C...   4.0   763.8175   
6  ACTA CRYSTALLOGRAPHICA SECTION F: STRUCTURAL B...   2.0   796.6350   
7                         ACTA DERMATO VENEREOLOGICA   1.0   653.9600   
8                                 ACTA DIABETOLOGICA   1.0  2336.2800   
9                                             ACTA F   1.0   754.9000   

                                                                 
          std      min        25%       50%        75%      max  
0         NaN  2379.54  2379.5400  2379.540  2379.5400  2379.54  
1  507.309560   947.07  1267.7600  1294.590  1294.7800  2286.73  
2         NaN  1186.80  1186.8000  1186.800  1186.8000  1186.80  
3   35.708892   642.89   655.5150   668.140   680.7650   693.39  
4         NaN   771.42   771.4200   771.420   771.4200   771.42  
5   12.064113   750.16   755.4250   765.460   773.8525   774.19  
6   15.605847   785.60   791.1175   796.635   802.1525   807.67  
7         NaN   653.96   653.9600   653.960   653.9600   653.96  
8         NaN  2336.28  2336.2800  2336.280  2336.2800  2336.28  
9         NaN   754.90   754.9000   754.900   754.9000   754.90